In [5]:
import pandas as pd
from simulations import  *
from keras.models import load_model
import numpy as np
import json
from keras.preprocessing.text import Tokenizer,tokenizer_from_json
from keras.utils import pad_sequences
import nltk

# 处理训练集

In [6]:
data=pd.read_excel("Problem_C_Data_Wordle.xlsx")
data=data.sort_values(by="Date")
data=data.reset_index().drop(columns="index")
data

,Date,Contest number,Word,Number of reported results,Number in hard mode,1 Try,2 Tries,3 Tries,4 Tries,5 Tries,6 Tries,Above 6 Tries
0,2022-01-07,202,slump,80630,1362,0.01,0.03,0.23,0.39,0.24,0.09,0.01
1,2022-01-08,203,crank,101503,1763,0.01,0.05,0.23,0.31,0.24,0.14,0.02
2,2022-01-09,204,gorge,91477,1913,0.01,0.03,0.13,0.27,0.30,0.22,0.04
3,2022-01-10,205,query,107134,2242,0.01,0.04,0.16,0.30,0.30,0.17,0.02
4,2022-01-11,206,drink,153880,3017,0.01,0.09,0.35,0.34,0.16,0.05,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...
354,2022-12-27,556,condo,20879,2012,0.00,0.02,0.17,0.35,0.29,0.14,0.03
355,2022-12-28,557,impel,20160,1937,0.00,0.03,0.21,0.40,0.25,0.09,0.01
356,2022-12-29,558,havoc,20001,1919,0.00,0.02,0.16,0.38,0.30,0.12,0.02
357,2022-12-30,559,molar,21204,1973,0.00,0.04,0.21,0.38,0.26,0.09,0.01


In [7]:
possible_words=data["Word"].to_csv("data/possible_words.txt",header=False,index=False)
optimalstart=["salet","reast","crate","trace","slate","crane"]

In [4]:
# import joblib
# # joblib.dump(resultall,"resultall.pkl")
# resultall=joblib.load("resultall.pkl")

In [8]:
def dataprocess(resultall,data):
    guesschainall=[]
    for q,k in enumerate(optimalstart):
        guesschain=[]
        for i in resultall[q]['game_results']:
            chain=i["guesses"]
            chain.append(i["answer"])
            guesschain.append(" ".join(chain))
        guesschainall.append(guesschain)

    reductionsall=[]
    for q,k in enumerate(optimalstart):
        reductions=[]
        for i in resultall[q]['game_results']:
            reductions.append(np.sum(i["reductions"]))
        reductionsall.append(reductions)

    scoresall=[]
    for q,k in enumerate(optimalstart):
        scores=[]
        for i in resultall[q]['game_results']:
            scores.append(i["score"])
        scoresall.append(scores)


    datainner=pd.DataFrame()
    datainner["Word"]=data["Word"].copy()
    for q,k in enumerate(optimalstart):
        datainner[k+" guesschain"]=guesschainall[q]
        datainner[k+" reductions"]=reductionsall[q]
        datainner[k+" score"]=scoresall[q]
    dataouter=pd.DataFrame()
    dataouter["Word"]=data["Word"].copy()
    RepeatN=[]
    for s in dataouter["Word"].values:
        RepeatN.append(len(s)-len(set(s)))
    dataouter["RepeatN"]=RepeatN
    WordCharacterall=nltk.pos_tag(data["Word"])
    WordCharacter=[]
    for s in WordCharacterall:
        WordCharacter.append(s[1])
    dataouter["WordCharacter"]=WordCharacter

    model = load_model('F:\\作业文件\\2023数学建模\\代码\\modeltrain\\主题分析\\saved_model\\情感2.h5')
    with open('F:\\作业文件\\2023数学建模\\代码\\modeltrain\\主题分析\\tokenizer.json', 'r', encoding='utf-8') as f1:
        tokenizer_config = json.load(f1)
    with open('F:\\作业文件\\2023数学建模\\代码\\modeltrain\\主题分析\\label_tokenizer_json.json', 'r', encoding='utf-8') as f2:
        label_tokenizer_config = json.load(f2)
    tokenizer = tokenizer_from_json(tokenizer_config)
    label_tokenizer = tokenizer_from_json(label_tokenizer_config)
    trunc_type = 'post'
    padding_type = 'post'
    max_len = 200
    for i in optimalstart:
        seq = np.array(pad_sequences(tokenizer.texts_to_sequences(datainner[i+" guesschain"]),truncating=trunc_type,padding=padding_type,maxlen=max_len))
        result = np.argmax(model.predict(seq),axis=1)[:,None].tolist()
        dataouter[i+" Topics"]=label_tokenizer.sequences_to_texts(result)
    model = load_model('F:\\作业文件\\2023数学建模\\代码\\modeltrain\\新闻主题分析\\saved_model\\情感2.h5')
    with open('F:\\作业文件\\2023数学建模\\代码\\modeltrain\\新闻主题分析\\tokenizer.json', 'r', encoding='utf-8') as f1:
        tokenizer_config = json.load(f1)
    with open('F:\\作业文件\\2023数学建模\\代码\\modeltrain\\新闻主题分析\\label_tokenizer_json.json', 'r', encoding='utf-8') as f2:
        label_tokenizer_config = json.load(f2)
    tokenizer = tokenizer_from_json(tokenizer_config)
    label_tokenizer = tokenizer_from_json(label_tokenizer_config)
    trunc_type = 'post'
    padding_type = 'post'
    max_len = 200
    for i in optimalstart:
        seq = np.array(pad_sequences(tokenizer.texts_to_sequences(datainner[i+" guesschain"]),truncating=trunc_type,padding=padding_type,maxlen=max_len))
        result = np.argmax(model.predict(seq),axis=1)[:,None].tolist()
        dataouter[i+" Description"]=label_tokenizer.sequences_to_texts(result)
    model = load_model('F:\\作业文件\\2023数学建模\\代码\\modeltrain\\正负情感分析\\saved_model\\情感2.h5')
    with open('F:\\作业文件\\2023数学建模\\代码\\modeltrain\\正负情感分析\\tokenizer.json', 'r', encoding='utf-8') as f1:
        tokenizer_config = json.load(f1)
    with open('F:\\作业文件\\2023数学建模\\代码\\modeltrain\\正负情感分析\\label_tokenizer_json.json', 'r', encoding='utf-8') as f2:
        label_tokenizer_config = json.load(f2)
    tokenizer = tokenizer_from_json(tokenizer_config)
    label_tokenizer = tokenizer_from_json(label_tokenizer_config)
    trunc_type = 'post'
    padding_type = 'post'
    max_len = 200
    for i in optimalstart:
        seq = np.array(pad_sequences(tokenizer.texts_to_sequences(datainner[i+" guesschain"]),truncating=trunc_type,padding=padding_type,maxlen=max_len))
        dataouter[i+" Attitude"]=model.predict(seq)[:,1]

    Word5 = data["Word"].apply(func = list)
    dfword = pd.DataFrame(data = [ii for ii in Word5.values],
                      columns=["word1","word2","word3","word4","word5"])
    datainner=datainner.join(dfword)
    return datainner,dataouter

In [ ]:
resultall=[]
for i in optimalstart:
    first_guess=i
    results, decision_map = simulate_games(
            first_guess=first_guess,
            priors=get_true_wordle_prior(),
            optimize_for_uniform_distribution=True,results_file="test.json",next_guess_map_file="test2.json",
            # shuffle=True,
            # brute_force_optimize=True,
            hard_mode=True
        )
    resultall.append(results)

In [10]:
datainner,dataouter=dataprocess(resultall,data)

12/12 [==============================] - 1s 62ms/step


In [11]:
datainner.to_csv("datainner.csv")
dataouter.to_csv("dataouter.csv")

In [12]:
data=pd.read_excel("Problem_C_Data_Wordle_test.xlsx")
data=data.sort_values(by="Date")
data=data.reset_index().drop(columns="index")
possible_words=data["Word"].to_csv("data/possible_words.txt",header=False,index=False)

In [ ]:
resultall=[]
for i in optimalstart:
    first_guess=i
    results, decision_map = simulate_games(
            first_guess=first_guess,
            priors=get_true_wordle_prior(),
            optimize_for_uniform_distribution=True,results_file="test.json",next_guess_map_file="test2.json",
            # shuffle=True,
            # brute_force_optimize=True,
            hard_mode=True
        )
    resultall.append(results)
datainner,dataouter=dataprocess(resultall,data)
datainner.to_csv("datainner_test.csv")
dataouter.to_csv("dataouter_test.csv")

In [ ]:
data=pd.read_excel("Problem_C_Data_Wordle_predict.xlsx")
data=data.sort_values(by="Date")
data=data.reset_index().drop(columns="index")
possible_words=data["Word"].to_csv("data/possible_words.txt",header=False,index=False)
resultall=[]
for i in optimalstart:
    first_guess=i
    results, decision_map = simulate_games(
            first_guess=first_guess,
            priors=get_true_wordle_prior(),
            optimize_for_uniform_distribution=True,results_file="test.json",next_guess_map_file="test2.json",
            # shuffle=True,
            # brute_force_optimize=True,
            hard_mode=True
        )
    resultall.append(results)
datainner,dataouter=dataprocess(resultall,data)
datainner.to_csv("datainner_pre.csv")
dataouter.to_csv("dataouter_pre.csv")